# Distributed debate

This example simulates a debate competition with three participant agents, including the affirmative side (Pro), the negative side (Con), and the adjudicator (Judge). 

Pro believes that AGI can be achieved using the GPT model framework, while Con contests it. Judge listens to both sides' arguments and provides an analytical judgment on which side presented a more compelling and reasonable case.

A fully distributed version can be found in `examples/distributed/distributed_debate.py`.
Here we provide a standalone multi-process version.

To install AgentScope, please follow the steps in [README.md](../README.md#installation).

First, we need to set the model configs of AgentScope.

In [ ]:
model_configs = [
    {
        "model_type": "openai",
        "config_name": "gpt-3.5-turbo",
        "model_name": "gpt-3.5-turbo",
        "api_key": "xxx",
        "organization": "xxx",
        "generate_args": {
            "temperature": 0.0,
        },
    },
    {
        "model_type": "openai",
        "config_name": "gpt-4",
        "model_name": "gpt-4",
        "api_key": "xxx",
        "organization": "xxx",
        "generate_args": {
            "temperature": 0.0,
        },
    }
]

Second, let's start the three agents in the debate. Note that each agent here will automatically starts a sub-process, and the `reply` method is executed within the sub-process.

In [ ]:
import agentscope
from agentscope.agents.dialog_agent import DialogAgent

agentscope.init(model_configs=model_configs)

pro_agent = DialogAgent(
    name="Pro",
    model_config_name="gpt-3.5-turbo",
    use_memory=True,
    sys_prompt="Assume the role of a debater who is arguing in favor of the proposition that AGI (Artificial General Intelligence) can be achieved using the GPT model framework. Construct a coherent and persuasive argument, including scientific, technological, and theoretical evidence, to support the statement that GPT models are a viable path to AGI. Highlight the advancements in language understanding, adaptability, and scalability of GPT models as key factors in progressing towards AGI.",
).to_dist()
con_agent = DialogAgent(
    name="Con",
    model_config_name="gpt-3.5-turbo",
    use_memory=True,
    sys_prompt="Assume the role of a debater who is arguing against the proposition that AGI can be achieved using the GPT model framework. Construct a coherent and persuasive argument, including scientific, technological, and theoretical evidence, to support the statement that GPT models, while impressive, are insufficient for reaching AGI. Discuss the limitations of GPT models such as lack of understanding, consciousness, ethical reasoning, and general problem-solving abilities that are essential for true AGI.",
).to_dist()
judge_agent = DialogAgent(
    name="Judge",
    model_config_name="gpt-3.5-turbo",
    use_memory=True,
    sys_prompt="Assume the role of an impartial judge in a debate where the affirmative side argues that AGI can be achieved using the GPT model framework, and the negative side contests this. Listen to both sides' arguments and provide an analytical judgment on which side presented a more compelling and reasonable case. Consider the strength of the evidence, the persuasiveness of the reasoning, and the overall coherence of the arguments presented by each side."
).to_dist()

Next, write the main debate competition process.
Note that we need to use `msghub` to ensure each agent in the debate knows the speeaches of all other agents.

In [ ]:
from agentscope.msghub import msghub
from agentscope.message import Msg
from agentscope.utils.logging_utils import logger

# Rules explained before the debate begins 
ANNOUNCEMENT = """
Welcome to the debate on whether Artificial General Intelligence (AGI) can be achieved using the GPT model framework. This debate will consist of three rounds. In each round, the affirmative side will present their argument first, followed by the negative side. After both sides have presented, the adjudicator will summarize the key points and analyze the strengths of the arguments.

The rules are as follows:

Each side must present clear, concise arguments backed by evidence and logical reasoning.
No side may interrupt the other while they are presenting their case.
After both sides have presented, the adjudicator will have time to deliberate and will then provide a summary, highlighting the most persuasive points from both sides.
The adjudicator's summary will not declare a winner for the individual rounds but will focus on the quality and persuasiveness of the arguments.
At the conclusion of the three rounds, the adjudicator will declare the overall winner based on which side won two out of the three rounds, considering the consistency and strength of the arguments throughout the debate.
Let us begin the first round. The affirmative side: please present your argument for why AGI can be achieved using the GPT model framework.
"""


"""Setup the main debate competition process"""
participants = [pro_agent, con_agent, judge_agent]
hint = Msg(name="System", content=ANNOUNCEMENT)
x = None
with msghub(participants=participants, announcement=hint):
    for _ in range(3):
        pro_resp = pro_agent(x)
        logger.chat(pro_resp)
        con_resp = con_agent(pro_resp)
        logger.chat(con_resp)
        x = judge_agent(con_resp)
        logger.chat(x)
    x = judge_agent(x)
    logger.chat(x)


Finally, just wait for the above code to run and watch the debate proceed.